In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Đọc dữ liệu từ file Excel
def analyze_web_profit_data(file_path):
    # Đọc dữ liệu - không cần chỉ định engine, pandas sẽ tự chọn
    df = pd.read_excel(file_path)
    
    # In thông tin cơ bản về dữ liệu
    print("Thông tin dữ liệu:")
    print(df.head())
    print("\nKích thước dữ liệu:", df.shape)
    
    # Phân tách dữ liệu thành hai nhóm: trước và sau
    before_data = df[df['Before.After'] == 'Before']['Profit']
    after_data = df[df['Before.After'] == 'After']['Profit']
    
    # Thống kê mô tả
    print("\nThống kê mô tả cho dữ liệu trước khi triển khai web mới:")
    print(before_data.describe())
    
    print("\nThống kê mô tả cho dữ liệu sau khi triển khai web mới:")
    print(after_data.describe())
    
    # Kiểm định t
    t_stat, p_value = stats.ttest_ind(before_data, after_data, equal_var=False)
    
    print("\nKết quả kiểm định t:")
    print(f"t-statistic: {t_stat:.4f}")
    print(f"p-value: {p_value:.4f}")
    
    # Kết luận kiểm định
    alpha = 0.05
    if p_value < alpha:
        conclusion = "Bác bỏ giả thuyết H0: Có sự khác biệt đáng kể về doanh thu."
    else:
        conclusion = "Không bác bỏ giả thuyết H0: Không có sự khác biệt đáng kể về doanh thu."
    
    print(f"\nVới mức ý nghĩa {alpha}, {conclusion}")
    
    # Tính khoảng tin cậy 95% cho sự khác biệt
    diff_mean = before_data.mean() - after_data.mean()
    se_diff = np.sqrt(before_data.var()/len(before_data) + after_data.var()/len(after_data))
    
    # Bậc tự do (Welch-Satterthwaite)
    df_welch = ((before_data.var()/len(before_data) + after_data.var()/len(after_data))**2) / \
               ((before_data.var()/len(before_data))**2/(len(before_data)-1) + 
                (after_data.var()/len(after_data))**2/(len(after_data)-1))
    
    # Giá trị tới hạn t
    t_critical = stats.t.ppf(1 - alpha/2, df_welch)
    
    # Khoảng tin cậy
    ci_lower = diff_mean - t_critical * se_diff
    ci_upper = diff_mean + t_critical * se_diff
    
    print(f"\nSự khác biệt trung bình: {diff_mean:.4f}")
    print(f"Khoảng tin cậy 95%: [{ci_lower:.4f}, {ci_upper:.4f}]")
    
    # Vẽ biểu đồ
    plot_results(before_data, after_data, p_value, alpha)
    
    return before_data, after_data, t_stat, p_value, conclusion

def plot_results(before_data, after_data, p_value, alpha):
    # Thiết lập figure
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    plt.subplots_adjust(hspace=0.3, wspace=0.3)
    
    # 1. Biểu đồ boxplot so sánh
    sns.boxplot(x=['Before']*len(before_data) + ['After']*len(after_data), 
                y=list(before_data) + list(after_data), 
                ax=axes[0, 0])
    axes[0, 0].set_title('So sánh phân phối doanh thu trước và sau khi triển khai web mới')
    axes[0, 0].set_xlabel('Thời điểm')
    axes[0, 0].set_ylabel('Doanh thu')
    
    # 2. Biểu đồ histogram
    axes[0, 1].hist(before_data, alpha=0.5, bins=15, label='Trước')
    axes[0, 1].hist(after_data, alpha=0.5, bins=15, label='Sau')
    axes[0, 1].set_title('Phân phối doanh thu')
    axes[0, 1].set_xlabel('Doanh thu')
    axes[0, 1].set_ylabel('Tần số')
    axes[0, 1].legend()
    
    # 3. Biểu đồ violin plot
    sns.violinplot(x=['Before']*len(before_data) + ['After']*len(after_data), 
                   y=list(before_data) + list(after_data), 
                   ax=axes[1, 0])
    axes[1, 0].set_title('Phân phối doanh thu dạng violin')
    axes[1, 0].set_xlabel('Thời điểm')
    axes[1, 0].set_ylabel('Doanh thu')
    
    # 4. Biểu đồ cột so sánh trung bình
    means = [before_data.mean(), after_data.mean()]
    errors = [before_data.std() / np.sqrt(len(before_data)), 
              after_data.std() / np.sqrt(len(after_data))]
    
    bars = axes[1, 1].bar(['Trước', 'Sau'], means, yerr=errors, capsize=10, color=['skyblue', 'lightgreen'])
    axes[1, 1].set_title('Doanh thu trung bình')
    axes[1, 1].set_ylabel('Doanh thu')
    
    # Thêm giá trị p-value và kết luận vào biểu đồ
    significance = "Có ý nghĩa thống kê" if p_value < alpha else "Không có ý nghĩa thống kê"
    plt.figtext(0.5, 0.01, f'p-value = {p_value:.4f} ({significance})', 
                ha='center', fontsize=12, bbox={"facecolor":"orange", "alpha":0.2, "pad":5})
    
    plt.show()

# Sử dụng hàm
if __name__ == "__main__":
    # Thay đổi đường dẫn đến file Excel của bạn
    file_path = "webprofit.xlsx"
    before_data, after_data, t_stat, p_value, conclusion = analyze_web_profit_data(file_path)
    
    print("\nKết thúc phân tích.")

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [5]:
# Thêm vào code của bạn - Kiểm định hoán vị (permutation test)
def permutation_test(x, y, n_permutations=10000):
    """Thực hiện kiểm định hoán vị để so sánh trung bình của hai mẫu."""
    # Tính thống kê ban đầu
    observed_diff = np.mean(x) - np.mean(y)
    
    # Gộp hai mẫu
    combined = np.concatenate([x, y])
    
    # Thực hiện hoán vị
    count = 0
    for i in range(n_permutations):
        # Xáo trộn dữ liệu
        np.random.shuffle(combined)
        
        # Chia lại thành hai mẫu
        perm_x = combined[:len(x)]
        perm_y = combined[len(x):]
        
        # Tính sự khác biệt trong hoán vị hiện tại
        perm_diff = np.mean(perm_x) - np.mean(perm_y)
        
        # Đếm số lần sự khác biệt hoán vị >= sự khác biệt quan sát được (theo giá trị tuyệt đối)
        if abs(perm_diff) >= abs(observed_diff):
            count += 1
    
    # Tính p-value
    p_value = count / n_permutations
    
    return p_value, observed_diff

# Áp dụng kiểm định hoán vị
perm_p_value, observed_diff = permutation_test(before_data.values, after_data.values)

print("\n=== KIỂM ĐỊNH HOÁN VỊ (PERMUTATION TEST) ===")
print(f"Sự khác biệt quan sát được: {observed_diff:.4f}")
print(f"P-value từ kiểm định hoán vị: {perm_p_value:.4f}")

if perm_p_value < alpha:
    perm_conclusion = "Bác bỏ giả thuyết H0: Có sự khác biệt đáng kể về doanh thu."
else:
    perm_conclusion = "Không bác bỏ giả thuyết H0: Không có sự khác biệt đáng kể về doanh thu."

print(f"Với mức ý nghĩa {alpha}, {perm_conclusion}")


=== KIỂM ĐỊNH HOÁN VỊ (PERMUTATION TEST) ===
Sự khác biệt quan sát được: -1.0904
P-value từ kiểm định hoán vị: 0.1678
Với mức ý nghĩa 0.05, Không bác bỏ giả thuyết H0: Không có sự khác biệt đáng kể về doanh thu.
